In [1]:
#default_exp protoclr
import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from unsupervised_meta_learning.proto_utils import prototypical_loss, get_prototypes

/home/ojas/anaconda3/envs/ai/lib/python3.9/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
#export

class ProtoCLR(pl.LightningModule):
    
    def __init__(self, encoder, n_support, n_query, lr_decay_step, lr_decay_rate,  lr=1e-3, distance='euclidean'):
        super().__init__()
        self.encoder = encoder
        self.n_support = n_support
        self.n_query = n_query
        
        self.distance = distance
        
        self.lr = lr
        self.lr_decay_rate = lr_decay_rate
        self.lr_decay_step = lr_decay_step
        
        self.automatic_optimization = False
    
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.encoder.parameters(), lr=self.lr)
        sch = torch.optim.lr_scheduler.StepLR(opt, step_size=self.lr_decay_step, gamma=self.lr_decay_rate)
        return opt, sch
    
    def training_step(self, batch, batch_idx):
        opt = self.optimizers()
        sch = self.lr_schedulers()
        
        opt.zero_grad()
        
        # Treat the first dim as way, the second as shots
        # and use the first shot as support (like 1-shot setting)
        # the remaining shots as query
        data = batch['data'].to(self.device) # [batch_size x ways x shots x image_dim]
        data = data.unsqueeze(0)
        # e.g. 50 images, 2 support, 2 query, miniImageNet: torch.Size([1, 50, 4, 3, 84, 84])
        batch_size = data.size(0)
        ways = data.size(1)
        
        # Divide into support and query shots 
        x_support = data[:,:,:self.n_support]
        x_support = x_support.reshape((batch_size, ways * self.n_support, *x_support.shape[-3:])) # e.g. [1,50*n_support,*(3,84,84)]
        x_query = data[:,:,self.n_support:]
        x_query = x_query.reshape((batch_size, ways * self.n_query, *x_query.shape[-3:])) # e.g. [1,50*n_query,*(3,84,84)]
        
        # Create dummy query labels
        y_query = torch.arange(ways).unsqueeze(0).unsqueeze(2) # batch and shot dim
        y_query = y_query.repeat(batch_size, 1, self.n_query)
        y_query = y_query.view(batch_size, -1).to(self.device)
        
        y_support = torch.arange(ways).unsqueeze(0).unsqueeze(2) # batch and shot dim
        y_support = y_support.repeat(batch_size, 1, self.n_support)
        y_support = y_support.view(batch_size, -1).to(self.device)
        
        # Extract features (first dim is batch dim)
        x = torch.cat([x_support, x_query], 1) # e.g. [1,50*(n_support+n_query),*(3,84,84)]
        z = self.encoder.forward(x)
        z_support = z[:,:ways * self.n_support] # e.g. [1,50*n_support,*(3,84,84)]
        z_query = z[:,ways * self.n_support:] # e.g. [1,50*n_query,*(3,84,84)]
        
        # Get prototypes
        if self.n_support == 1:
            z_proto = z_support # in 1-shot the prototypes are the support samples
        else:
            z_proto = get_prototypes(z_support, y_support, ways)
            
        loss, accuracy = prototypical_loss(z_proto, z_query, y_query,
                                           distance=self.distance)
        
        self.manual_backward(loss)
        opt.step()
        sch.step()
        
        self.log_dict({
            'loss': loss.item(),
            'train_accuracy': accuracy.item()
        })
        
        return loss, accuracy
        
    